In [81]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from functools import partial
from multiprocessing import Pool
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

%matplotlib inline
%config IPCompleter.greedy=True
plt.rcParams["figure.figsize"] = (14, 8)

Analyze = False

In [82]:
class Params:
    def __init__(self,Func,Y):
        if Func == 'Full':
            epochs = 1000
            reps = 100
        else:
            epochs = 50
            reps = 3
        if Y == 'co2_flux':
            N = 23
            T = 0
        else:
            N = 82
            T = 2
        self.params = {}
        self.params['T'] = T
        self.params['N'] = N
        self.params['reps'] = reps
        self.params['epochs'] = epochs
        self.params['Y'] = Y
        self.params['proc']=3


In [83]:
def LSTM_Model(Neurons,batch_size,time_steps,inputs,lr=1e-4,Memory=.9):
    import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import LSTM
    from keras.wrappers.scikit_learn import KerasRegressor
    from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
#     from keras import backend as K
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = Memory
    session = tf.Session(config=config)
    model = Sequential()
    model.add(LSTM(Neurons, input_shape=(time_steps,inputs),stateful = False))
    model.add(Dense(1))
#     NUM_GPU = 1 # or the number of GPUs available on machine # - Add if using MXNET
#     adam = keras.optimizers.Adam(lr = lr)
#     gpu_list = []
#     for i in range(NUM_GPU): gpu_list.append('gpu(%d)' % i)
    model.compile(loss='mean_squared_error', optimizer='adam')#,context=gpu_list) # - Add if using MXNET
    return(model)

def Train_Steps(epochs,Neurons,X_train,X_test,X_val,y_train,y_test,y_val,i,Memory=None,
                GapFill=None,FullX=None):
    
    np.random.seed(i)
    from keras import backend as K
    Scorez=[]
    lr = 1e-3
    Mod = LSTM_Model(Neurons,X_train.shape[0],X_train.shape[1],X_train.shape[2],lr=lr,Memory=Memory)
    killscore=0
    killmax = 10
    e = 0
    udate = 3
    while killscore < killmax and e < epochs:
        Mod.fit(X_train,y_train,batch_size=X_train.shape[0], nb_epoch=1,shuffle=True,verbose=0)
        old_weights = Mod.get_weights()
        Y = Mod.predict(X_test,batch_size =X_test.shape[0])
        score = metrics.mean_squared_error(y_test,Y)
        Scorez.append(score)
        if e == 0:
            score_min=score
            min_weights=old_weights
        elif score < score_min:
            score_min = score
            min_weights=old_weights
            killscore = 0
        else:
            killscore +=1
        if killscore == math.floor(killmax/2):
            K.set_value(Mod.optimizer.lr, 0.5 * K.get_value(Mod.optimizer.lr))
        Mod.reset_states()
        e +=1
    Mod.set_weights(min_weights)
    Yval = Mod.predict(X_val,batch_size = X_val.shape[0])
    MSE = (metrics.mean_squared_error(y_val,Yval))
    Scorez=np.asanyarray(Scorez)
    if GapFill == None:
        return(MSE)
    else:
        return(Mod.predict(FullX,batch_size=FullX.shape[0]))
           

In [84]:
class ReadStandardTime:
    def __init__(self,Path):
        self.Path=Path
        self.Master = pd.read_csv(Path,delimiter = ',',header = 0,na_values = -9999)
        self.Master = self.Master.set_index(pd.DatetimeIndex(pd.to_datetime(self.Master['datetime'])))
        self.Master['DOY'] = self.Master.index.dayofyear*1.0
        self.Master['HR'] = self.Master.index.hour*1.0
        
    def Scale(self,y_var,X_vars):
        self.y_var = y_var
        self.Data = self.Master[np.isfinite(self.Master[y_var])]
                           
        self.Data = self.Data.interpolate().bfill()
        self.Data = self.Data.interpolate().ffill()
        self.y = self.Data[y_var].values
        YStandard = StandardScaler()
        self.YScaled = YStandard.fit(self.y.reshape(-1, 1))
        Yscale = self.YScaled.transform(self.y.reshape(-1, 1))
        self.y = np.ndarray.flatten(Yscale)
        self.Ytru = self.YScaled.inverse_transform(self.y.reshape(-1,1))
        X = self.Data[X_vars]
        self.input_shape = len(X_vars)
        XStandard = StandardScaler()
        self.XScaled= XStandard.fit(X)
        self.X = self.XScaled.transform(X)
        Filling = self.Master[X_vars]
        Filling = Filling.interpolate().bfill()
        Filling = Filling.interpolate().ffill()
        XStandard = StandardScaler()
        self.XFillScaled= XStandard.fit(Filling)
        self.X_fill = self.XScaled.transform(Filling)
        
    def TimeShape(self,rolls):
        X1 = self.X
        Xf = self.X_fill
        self.X_time = np.zeros(shape = (X1.shape[0],rolls+1,X1.shape[1]))
        self.X_time[:,0,:] = X1
        self.X_ftime = np.zeros(shape = (Xf.shape[0],rolls+1,Xf.shape[1]))
        self.X_ftime[:,0,:] = Xf
        if rolls > 0:
            for roll in range(0,rolls):
                X2 = np.roll(X1,(roll+1),axis=0)
                X2f = np.roll(Xf,(roll+1),axis=0)
                self.X_time[:,roll+1,:] = X2
                self.X_ftime[:,roll+1,:] = Xf
        self.X_time = self.X_time[rolls+1:,:,:]
        self.X_ftime = self.X_ftime[rolls+1:,:,:]
        self.y_time = self.y[rolls+1:]
        self.y_ftime = self.y[rolls+1:]
        self.TimeSteps = rolls+1
        
    def Fill(self,Y_Pred):
        Y_fill = self.YScaled.inverse_transform(Y_Pred.reshape(-1,1))
        nanz = np.zeros(shape=(self.TimeSteps,1))
        nanz[:,:] = np.nan
        Y_Pred = np.concatenate((nanz,Y_fill),axis=0).reshape(-1,1)
        print(self.Master.shape,Y_Pred.shape)
        self.Master['TempFill'] = Y_Pred
        self.Master[self.y_var+'_Filled'] = self.Master[self.y_var].fillna(self.Master['TempFill'])
#         self.Master.drop('TempFill',inplace=True)
        self.Master.to_csv(self.Path)

In [85]:
def Combos(Model,L,factor=None):
    Models=list()
    for c in combinations(Model,L):
        c = list(c)
        if factor == None:
            Models.append(c)
        else:
            if factor in c:
                Models.append(c)
    return(Models)

def TTV_Split(i,Memory,X,y,params,GapFill,FullX):
    X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.1, random_state=i)
    X_train,X_val,y_train,y_val=train_test_split(X_train,y_train, test_size=0.11, random_state=i)
    if GapFill==None:
        return(Train_Steps(params['epochs'],params['N'],X_train,X_test,X_val,y_train,y_test,y_val,i=i,Memory=Memory))
    else:
        return(Train_Steps(params['epochs'],params['N'],X_train,X_test,X_val,y_train,y_test,y_val,i=i,Memory=Memory,
                              GapFill=GapFill,FullX=FullX))

# Model = ['Temp_2_5_1','u*','air_pressure','Table_1','NR_Wm2_Avg','AirTC_Avg','RH','DOY']


Model = ["DOY","H","air_pressure","wind_speed","wind_dir","u*","Hz","AirTC_Avg","RH_Samp","Rain_mm_Tot","NR_Wm2_Avg",
         "PPFD_Avg","Temp_2_5_1","Temp_15_1","Temp_5_1","Temp_2_5_2","Temp_15_2","Temp_5_2","Depth_1","Active_Layer_1",
         "Table_1","Depth_2","Active_Layer_2"]

Time = time.time()

Path = 'FI_Data_W_BL.csv' 
def RunReps(Model,params,pool = None,GapFill=None):
    RST = ReadStandardTime(Path)
    offset = 5/params['proc']
    Memory = (math.floor(100/params['proc'])- offset) * .01
    MSE = []
    RST.Scale(params['Y'],Model)
    RST.TimeShape(params['T'])
    y = RST.y_time*1.0
    X = RST.X_time*1.0
    FullX = RST.X_ftime
    MSE = []
    Y_Pred = []
    if __name__=='__main__'and params['proc'] != 1:
        for i,res in enumerate(pool.imap(partial(TTV_Split,Memory=Memory,X=X,y=y,params=params,GapFill=GapFill,FullX=FullX),
                                         range(params['reps']))):
            if GapFill == None:
                MSE.append(res)
            else:
                Y_Pred.append(res)
                
    else:
        for i in range(params['reps']):
            res = TTV_Split(i,Memory,X,y,params,GapFill=GapFill,FullX=FullX)
            if GapFill == None:
                MSE.append(res)
            else:
                Y_Pred.append(res)
    if GapFill == None:
        MSE = np.asanyarray(MSE)
        return(MSE)
    else:
        Y_Pred = np.asanyarray(Y_Pred).mean(axis=0)
        RST.Fill(Y_Pred)
        print(Y_Pred.shape)
        return('kitty')
    
Scores = []
L = len(Model)-1
Models = Combos(Model,L)


## Gap Filling - Full Model

In [86]:
pool.close()

if __name__=='__main__'and params['proc'] != 1:
    pool = Pool(processes=params['proc'],maxtasksperchild=60)
else:pool=None
    
Fluxes = ['co2_flux','ch4_flux']
for flux in Fluxes:
    rpms = Params('temp',flux)
    params=rpms.params
    print(params)
    Results = RunReps(Model,params,pool,GapFill=True)
# Score.append(Results.mean())
# Score = np.asanyarray(Score)

# Worst = Models[np.where(Score == Score.max())[0][0]]
# Best = Models[np.where(Score == Score.min())[0][0]]
# Scores.append(Score)
# L-=1
# if L >1:
#     print(Best)
#     print(Worst)
#     for factor in Best:
#         print(factor)
#         if factor not in Worst:
#             print('Best Factor: ', factor)
#             Models = Combos(Best,L,factor)
# else:
#     Models=[Best]
# print()
# print('Best Factor: ',Best)
# print(time.time()-Time)
if __name__=='__main__'and params['proc'] != 1:
    pool.close()
        

{'epochs': 50, 'N': 23, 'Y': 'co2_flux', 'reps': 3, 'proc': 3, 'T': 0}


Using TensorFlow backend.
Using TensorFlow backend.
Using TensorFlow backend.


(4021, 261) (4021, 1)
(4020, 1)
{'epochs': 50, 'N': 82, 'Y': 'ch4_flux', 'reps': 3, 'proc': 3, 'T': 2}
(4021, 262) (4021, 1)
(4018, 1)


## Factor Analysis

In [87]:
if Analyze == True:
    if __name__=='__main__'and params['proc'] != 1:
        pool = Pool(processes=params['proc'],maxtasksperchild=60)
    else:pool=None
    while L > 0:
        print('Level: ', L)

        print()
        print(Models)
        print()

        Score = []
        for i in range(len(Models)):

            Results = RunReps(Models[i],params,pool)
            Score.append(Results.mean())
        Score = np.asanyarray(Score)

        Worst = Models[np.where(Score == Score.max())[0][0]]
        Best = Models[np.where(Score == Score.min())[0][0]]
        Scores.append(Score)
        L-=1
        if L >1:
            print(Best)
            print(Worst)
            for factor in Best:
                print(factor)
                if factor not in Worst:
                    print('Best Factor: ', factor)
                    Models = Combos(Best,L,factor)
        else:
            Models=[Best]
        print()
    print('Best Factor: ',Best)
    print(time.time()-Time)
    if __name__=='__main__'and params['proc'] != 1:
        pool.close()

    Scores = np.asanyarray(Scores)
    plt.boxplot(Scores)
    plt.show()

NameError: name 'Analyze' is not defined